In [1]:
# Machine Learning Pipeline Execution
# Este notebook ejecuta secuencialmente todos los pasos del pipeline ML

# Importaciones necesarias
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Asegurarse de que podemos importar desde la raíz del proyecto
# (ajusta esto si es necesario según la estructura de tu proyecto)
sys.path.append('..')



In [2]:
import os
print(f"Directorio de trabajo actual: {os.getcwd()}")
# Si necesitas cambiar el directorio:
#os.chdir('/ruta/a/tu/proyecto')

Directorio de trabajo actual: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\notebooks


In [3]:
import os
import shutil
import sys

# 1. Detectar la estructura del proyecto de forma dinámica
# Obtener el directorio actual
current_dir = os.getcwd()
print(f"Directorio actual: {current_dir}")

# Determinar la ruta al directorio raíz del proyecto
# Asumiendo que estamos en un notebook dentro del proyecto
if 'SP500_INDEX_Analisis' in current_dir:
    # Buscar el directorio raíz del proyecto (subiendo hasta encontrar SP500_INDEX_Analisis)
    path_parts = current_dir.split('SP500_INDEX_Analisis')
    project_root = os.path.join(path_parts[0], 'SP500_INDEX_Analisis')
else:
    # Si no estamos en la estructura esperada, usamos el directorio actual
    project_root = current_dir

print(f"Raíz del proyecto detectada: {project_root}")

# 2. Definir rutas relativas al project_root
pipelines_dir = os.path.join(project_root, "pipelines")
ml_dir = os.path.join(pipelines_dir, "ml")
config_path = os.path.join(pipelines_dir, "Data Engineering.xlsx")

# Verificar si el archivo de configuración existe
if not os.path.exists(config_path):
    print(f"ADVERTENCIA: El archivo de configuración no se encuentra en: {config_path}")
else:
    print(f"Archivo de configuración encontrado en: {config_path}")

# 3. Asegurar que existan los directorios de datos
data_root = os.path.join(project_root, "Data", "0_raw")
if not os.path.exists(data_root):
    os.makedirs(data_root, exist_ok=True)
    print(f"Creado directorio de datos: {data_root}")

# 4. Configurar path para importaciones
sys.path.append(project_root)
sys.path.append(ml_dir)

# 5. Ejecutar los procesadores de datos
try:
    from step_0_preprocess import run_economic_data_processor, ejecutar_myinvestingreportnormal, run_fred_data_processor, ejecutar_otherdataprocessor
    
    print("\nEjecutando procesadores de datos con rutas relativas:")
    print("-------------------------------------------")
    
    # Ejecutar cada procesador con la ruta correcta del archivo de configuración
    run_economic_data_processor(config_file=config_path, data_root=data_root)
    ejecutar_myinvestingreportnormal(config_file=config_path, data_root=data_root)
    run_fred_data_processor(config_file=config_path, data_root=data_root)
    ejecutar_otherdataprocessor(config_file=config_path, data_root=data_root)
    
except ImportError as e:
    print(f"Error al importar funciones de preprocesamiento: {e}")
    print("\nCreando y ejecutando un script temporal con las rutas relativas...")
    
    # Crear un script temporal con las rutas relativas
    temp_script = os.path.join(current_dir, "temp_preprocess.py")
    script_content = f"""
import os
import sys

# Configurar rutas relativas
project_root = r"{project_root}"
sys.path.append(project_root)
sys.path.append(os.path.join(project_root, "pipelines", "ml"))

from pipelines.ml.step_0_preprocess import run_economic_data_processor, ejecutar_myinvestingreportnormal, run_fred_data_processor, ejecutar_otherdataprocessor

# Ejecutar con las rutas relativas
config_path = os.path.join(project_root, "pipelines", "Data Engineering.xlsx")
data_root = os.path.join(project_root, "Data", "0_raw")

# Verificar que el archivo de configuración existe
if not os.path.exists(config_path):
    print(f"ERROR: No se encuentra el archivo de configuración en: {{config_path}}")
    sys.exit(1)
else:
    print(f"Usando archivo de configuración: {{config_path}}")

# Ejecutar los procesadores
run_economic_data_processor(config_file=config_path, data_root=data_root)
ejecutar_myinvestingreportnormal(config_file=config_path, data_root=data_root)
run_fred_data_processor(config_file=config_path, data_root=data_root)
ejecutar_otherdataprocessor(config_file=config_path, data_root=data_root)
"""
    with open(temp_script, "w") as f:
        f.write(script_content)
    
    # Ejecutar el script temporal
    print(f"Ejecutando script temporal: {temp_script}")
    %run {temp_script}

2025-05-12 13:15:52,062 - INFO - ================================================================================
2025-05-12 13:15:52,063 - INFO - INICIANDO PROCESO: EconomicDataProcessor
2025-05-12 13:15:52,064 - INFO - Archivo de configuración: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\pipelines\Data Engineering.xlsx
2025-05-12 13:15:52,065 - INFO - Directorio raíz de datos: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\Data\0_raw
2025-05-12 13:15:52,065 - INFO - Fecha y hora: 2025-05-12 13:15:52
2025-05-12 13:15:52,066 - INFO - ================================================================================
2025-05-12 13:15:52,067 - INFO - Leyendo archivo de configuración...


Directorio actual: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\notebooks
Raíz del proyecto detectada: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis
Archivo de configuración encontrado en: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\pipelines\Data Engineering.xlsx

Ejecutando procesadores de datos con rutas relativas:
-------------------------------------------


2025-05-12 13:15:52,456 - INFO - Se encontraron 21 configuraciones para procesar
2025-05-12 13:15:52,458 - INFO - 
Procesando: US_ISM_Manufacturing (business_confidence)
2025-05-12 13:15:52,458 - INFO - - Archivo: US_ISM_Manufacturing.xlsx
2025-05-12 13:15:52,460 - INFO - - Columna TARGET: ACTUAL
2025-05-12 13:15:52,460 - INFO - - Ruta encontrada: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\Data\0_raw\business_confidence\US_ISM_Manufacturing.xlsx
2025-05-12 13:15:52,487 - INFO - - Filas encontradas: 136
2025-05-12 13:15:52,495 - INFO - Preferencia de dayfirst para c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\Data\0_raw\business_confidence\US_ISM_Manufacturing.xlsx: True
2025-05-12 13:15:52,564 - WARNING - No se encontró 'ACTUAL', se usará 'Actual'
2025-05-12 13:15:52,573 - INFO - - Valores no nulos en TARGET: 136
2025-05-12 13:15:52,574 - INFO - - Periodo: 2014-01-02 a 2025-04-01
2025-05-

In [50]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Paso 0: Preprocesamiento inicial
print("Ejecutando paso 0: Preprocesamiento inicial")
%run ../pipelines/ml/step_0_preprocess.py

2025-05-12 13:36:55,236 - INFO - ================================================================================
2025-05-12 13:36:55,237 - INFO - INICIANDO PROCESO: EconomicDataProcessor
2025-05-12 13:36:55,238 - INFO - Archivo de configuración: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\pipelines/Data Engineering.xlsx
2025-05-12 13:36:55,239 - INFO - Directorio raíz de datos: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\data/0_raw
2025-05-12 13:36:55,240 - INFO - Fecha y hora: 2025-05-12 13:36:55
2025-05-12 13:36:55,240 - INFO - ================================================================================
2025-05-12 13:36:55,241 - INFO - Leyendo archivo de configuración...
2025-05-12 13:36:55,287 - INFO - Se encontraron 21 configuraciones para procesar
2025-05-12 13:36:55,288 - INFO - 
Procesando: US_ISM_Manufacturing (business_confidence)
2025-05-12 13:36:55,289 - INFO - - Archivo:

Ejecutando paso 0: Preprocesamiento inicial


2025-05-12 13:36:55,326 - INFO - - Filas encontradas: 136
2025-05-12 13:36:55,334 - INFO - Preferencia de dayfirst para c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\data/0_raw\business_confidence\US_ISM_Manufacturing.xlsx: True
2025-05-12 13:36:55,395 - WARNING - No se encontró 'ACTUAL', se usará 'Actual'
2025-05-12 13:36:55,397 - INFO - - Valores no nulos en TARGET: 136
2025-05-12 13:36:55,398 - INFO - - Periodo: 2014-01-02 a 2025-04-01
2025-05-12 13:36:55,398 - INFO - - Cobertura: 100.00%
2025-05-12 13:36:55,399 - INFO - 
Procesando: US_ISM_Services (business_confidence)
2025-05-12 13:36:55,400 - INFO - - Archivo: US_ISM_Services.xlsx
2025-05-12 13:36:55,400 - INFO - - Columna TARGET: ACTUAL
2025-05-12 13:36:55,401 - INFO - - Ruta encontrada: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\data/0_raw\business_confidence\US_ISM_Services.xlsx
2025-05-12 13:36:55,432 - INFO - - Filas encontrad

Proceso completado exitosamente


2025-05-12 13:36:59,109 - INFO - Ejemplos de fechas convertidas para Australia_10Y_Bond: [Timestamp('2025-04-30 00:00:00'), Timestamp('2025-04-29 00:00:00'), Timestamp('2025-04-28 00:00:00'), Timestamp('2025-04-24 00:00:00'), Timestamp('2025-04-23 00:00:00')]
2025-05-12 13:36:59,112 - INFO - Formato numérico detectado para Australia_10Y_Bond: americano
2025-05-12 13:36:59,118 - INFO - Para Australia_10Y_Bond (columna Date), la fecha mínima es 2014-01-01 00:00:00 y la fecha máxima es 2025-04-30 00:00:00
2025-05-12 13:36:59,119 - INFO - - Australia_10Y_Bond: 3832 filas procesadas, periodo: 2014-01-01 a 2025-04-30
2025-05-12 13:36:59,120 - INFO - 
Procesando: Italy_10Y_Bond (bond)
2025-05-12 13:36:59,121 - INFO - - Archivo: Italy_10Y_Bond
2025-05-12 13:36:59,122 - INFO - - Columna TARGET: PRICE
2025-05-12 13:36:59,122 - INFO - - Ruta encontrada: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\data/0_raw\bond\Italy_10Y_Bond.csv
2025-05-12 13:36

...

Proceso completado exitosamente


2025-05-12 13:38:45,966 - INFO - Se encontraron 4 configuraciones para procesar con fuente 'Other'
2025-05-12 13:38:45,970 - INFO - 
Resumen de configuraciones a procesar:
2025-05-12 13:38:45,974 - INFO - - US_Empire_State_Index (Tipo Macro: business_confidence, TARGET: nan)
2025-05-12 13:38:45,978 - INFO - - AAII_Investor_Sentiment (Tipo Macro: consumer_confidence, TARGET: nan)
2025-05-12 13:38:45,981 - INFO - - Put_Call_Ratio_SPY (Tipo Macro: consumer_confidence, TARGET: nan)
2025-05-12 13:38:45,984 - INFO - - Chicago_Fed_NFCI (Tipo Macro: leading_economic_index, TARGET: ANFCI)
2025-05-12 13:38:45,989 - INFO - 
Procesando: US_Empire_State_Index (business_confidence)
2025-05-12 13:38:45,992 - INFO - - Columna TARGET: nan
2025-05-12 13:38:46,000 - INFO - - Tipo de preprocesamiento: Normal (Historical Tables)
2025-05-12 13:38:46,005 - INFO - Procesando US_Empire_State_Index manualmente
2025-05-12 13:38:46,010 - INFO - - Archivo de entrada: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Pr

Proceso completado exitosamente


2025-05-12 13:39:07,589 - INFO - Archivo Excel cargado exitosamente desde C:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\Data\1_preprocess\MERGEDEXCELS.xlsx
2025-05-12 13:39:07,591 - INFO - Iniciando la categorización de columnas...
2025-05-12 13:39:07,600 - INFO - Proceso completado exitosamente.


columnas encontradas para Sin categoría:
fecha

columnas encontradas para bond:
PRICE_Australia_10Y_Bond_bond
PRICE_Italy_10Y_Bond_bond
PRICE_Japan_10Y_Bond_bond
PRICE_UK_10Y_Bond_bond
PRICE_Germany_10Y_Bond_bond
PRICE_Canada_10Y_Bond_bond
PRICE_China_10Y_Bond_bond
DGS10_US_10Y_Treasury_bond
DGS2_US_2Y_Treasury_bond
AAA_Corporate_Bond_AAA_Spread_bond
BAA10YM_Corporate_Bond_BBB_Spread_bond
BAMLH0A0HYM2_High_Yield_Bond_Spread_bond

columnas encontradas para commodities:
PRICE_CrudeOil_WTI_commodities
PRICE_Gold_Spot_commodities
PRICE_Silver_Spot_commodities
PRICE_Copper_Futures_commodities
PRICE_Platinum_Spot_commodities

columnas encontradas para exchange_rate:
PRICE_EUR_USD_Spot_exchange_rate
PRICE_GBP_USD_Spot_exchange_rate
PRICE_JPY_USD_Spot_exchange_rate
PRICE_CNY_USD_Spot_exchange_rate
PRICE_AUD_USD_Spot_exchange_rate
PRICE_CAD_USD_Spot_exchange_rate
PRICE_MXN_USD_Spot_exchange_rate
PRICE_EUR_GBP_Cross_exchange_rate

columnas encontradas para index_pricing:
PRICE_S&P500_Index_index

In [6]:
# Paso 1: Fusión de archivos Excel
print("Ejecutando paso 1: Fusión de archivos Excel")
%run ../pipelines/ml/step_1_merge_excels.py

2025-05-12 13:40:27,084 - INFO - === INICIANDO PROCESO DE COMBINACIÓN DE ARCHIVOS EXCEL ===
2025-05-12 13:40:27,087 - INFO - Se encontraron 4 archivos Excel en la carpeta.
2025-05-12 13:40:27,087 - INFO - Cargando archivo: datos_economicos_normales_procesados.xlsx


Ejecutando paso 1: Fusión de archivos Excel


2025-05-12 13:40:27,848 - INFO - Archivo datos_economicos_normales_procesados.xlsx cargado en 0.76s con 4140 filas.
2025-05-12 13:40:27,849 - INFO - Archivo datos_economicos_normales_procesados.xlsx cargado correctamente con 4140 filas
2025-05-12 13:40:27,851 - INFO - Filtrado por fecha completado en 0.00s: 4140 → 4104 filas.
2025-05-12 13:40:27,853 - INFO - Después de filtrar por fecha: 4104 filas
2025-05-12 13:40:27,853 - INFO - Cargando archivo: datos_economicos_other_procesados.xlsx
2025-05-12 13:40:28,548 - INFO - Archivo datos_economicos_other_procesados.xlsx cargado en 0.69s con 8675 filas.
2025-05-12 13:40:28,549 - INFO - Archivo datos_economicos_other_procesados.xlsx cargado correctamente con 8675 filas
2025-05-12 13:40:28,551 - INFO - Filtrado por fecha completado en 0.00s: 8675 → 4104 filas.
2025-05-12 13:40:28,553 - INFO - Después de filtrar por fecha: 4104 filas
2025-05-12 13:40:28,554 - INFO - Cargando archivo: datos_economicos_procesados_cp.xlsx
2025-05-12 13:40:28,846 -

In [7]:
# Paso 2: Generación de categorías
print("Ejecutando paso 2: Generación de categorías")
%run ../pipelines/ml/step_2_generate_categories.py



Ejecutando paso 2: Generación de categorías


2025-05-12 13:40:41,906 - INFO - Archivo cargado correctamente. Dimensiones: (4104, 85)
2025-05-12 13:40:41,909 - WARNING - Las siguientes columnas no se encontraron en el archivo: ['Denmark_Car_Resistrations', 'US_Car_Registrations', 'SouthAfrica_Car_Registrations', 'United_Kingdom_Car_Registrations', 'Spain_Car_Registrations', 'Singapore_NonOil_Exports', 'Japan_M2_MoneySupply', 'China_M2_MoneySupply', 'US_Industrial_Production', 'UK_Retail_Sales']
2025-05-12 13:40:41,909 - INFO - No se encontraron columnas para renombrar.
2025-05-12 13:40:41,956 - INFO - Resultados detallados de categorización guardados en: c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\data\1_preprocess\DIAGNOSTICO_CATEGORIAS.xlsx
2025-05-12 13:40:41,958 - INFO - Resumen de categorización:
2025-05-12 13:40:41,958 - INFO -   - Sin categoría: 1 columnas
2025-05-12 13:40:41,959 - INFO -   - bond: 11 columnas
2025-05-12 13:40:41,960 - INFO -   - business_confidence: 7 colum

In [8]:
# Paso 3: Limpieza de columnas
print("Ejecutando paso 3: Limpieza de columnas")
%run ../pipelines/ml/step_3_clean_columns.py



Ejecutando paso 3: Limpieza de columnas


2025-05-12 13:45:02,391 - INFO - Archivo cargado correctamente. Columnas: 85
2025-05-12 13:45:02,393 - INFO - Se modificaron 5 nombres de columnas.
2025-05-12 13:45:02,394 - INFO - Renombrando: 'DNKSLRTCR03GPSAM_Denmark_Car_Registrations_MoM_car_registrations' -> 'DNKSLRTCR03GPSAM_Denmark_Car_Registrations_MoM'
2025-05-12 13:45:02,394 - INFO - Renombrando: 'USASLRTCR03GPSAM_US_Car_Registrations_MoM_car_registrations' -> 'USASLRTCR03GPSAM_US_Car_Registrations_MoM'
2025-05-12 13:45:02,394 - INFO - Renombrando: 'ZAFSLRTCR03GPSAM_SouthAfrica_Car_Registrations_MoM_car_registrations' -> 'ZAFSLRTCR03GPSAM_SouthAfrica_Car_Registrations_MoM'
2025-05-12 13:45:02,395 - INFO - Renombrando: 'GBRSLRTCR03GPSAM_United_Kingdom_Car_Registrations_MoM_car_registrations' -> 'GBRSLRTCR03GPSAM_United_Kingdom_Car_Registrations_MoM'
2025-05-12 13:45:02,396 - INFO - Renombrando: 'ESPSLRTCR03GPSAM_Spain_Car_Registrations_MoM_car_registrations' -> 'ESPSLRTCR03GPSAM_Spain_Car_Registrations_MoM'
2025-05-12 13:45:07

In [9]:
# Paso 4: Transformación de características
print("Ejecutando paso 4: Transformación de características")
%run ../pipelines/ml/step_4_transform_features.py


Ejecutando paso 4: Transformación de características
✅ Archivo Excel cargado exitosamente.
🧪 Primeras filas del archivo:
                    0                              1   \
0                fecha  PRICE_Australia_10Y_Bond_bond   
1        Sin categoría                           bond   
2  2014-01-01 00:00:00                          4.289   
3  2014-01-02 00:00:00                          4.331   
4  2014-01-03 00:00:00                          4.344   

                          2                          3   \
0  PRICE_Italy_10Y_Bond_bond  PRICE_Japan_10Y_Bond_bond   
1                       bond                       bond   
2                        NaN                        NaN   
3                        NaN                        NaN   
4                        NaN                        NaN   

                       4                            5   \
0  PRICE_UK_10Y_Bond_bond  PRICE_Germany_10Y_Bond_bond   
1                    bond                         bond   
2      

In [ ]:

# Paso 5: Eliminación de relaciones
print("Ejecutando paso 5: Eliminación de relaciones")
%run ../pipelines/ml/step_5_remove_relations.py

Ejecutando paso 5: Eliminación de relaciones


2025-05-12 13:47:37,866 - INFO - Iniciando proceso de selección y filtrado de features...
2025-05-12 13:47:46,907 - INFO - Datos cargados: 2932 filas, 474 columnas
2025-05-12 13:47:46,908 - INFO - Target identificado: ICSA_US_Initial_Jobless_Claims_unemployment_rate
2025-05-12 13:47:47,182 - INFO - Features constantes eliminadas: 2
2025-05-12 13:47:47,183 - INFO - Ejemplos: ['id', 'log_diff_AAA_Corporate_Bond_AAA_Spread_bond']...
2025-05-12 13:47:47,186 - INFO - Features numéricas para análisis: 470
2025-05-12 13:47:48,594 - INFO - Features correlacionadas eliminadas: 255
2025-05-12 13:47:48,595 - INFO - Ejemplos: ['EMA_5_PRICE_CrudeOil_WTI_commodities', 'PRICE_UK_10Y_Bond_bond', 'rolling_std_PRICE_Platinum_Spot_commodities', 'MA_20_PRICE_Shanghai_Composite_index_pricing', 'WALCL_Fed_Balance_Sheet_economics']...
2025-05-12 13:48:20,519 - INFO - Eliminando 'PAYEMS_US_Nonfarm_Payrolls_unemployment_rate' por VIF = 388778.60
2025-05-12 13:48:52,478 - INFO - Eliminando 'log_PRICE_JPY_USD_Sp

In [11]:
!pip install catboost
!pip install feature_engine
!pip install pandas_market_calendars

In [12]:
# Paso 6: Selección FPI
print("Ejecutando paso 6: Selección FPI")
%run ../pipelines/ml/step_6_fpi_selection.py



Ejecutando paso 6: Selección FPI


2025-05-12 15:28:36,821 - INFO - Usando el archivo más reciente: C:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\data\2_processed\ULTIMO_S&P500_final.xlsx
2025-05-12 15:28:36,822 - INFO - La salida se guardará en: C:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\data\3_trainingdata\ULTIMO_S&P500_final_FPI.xlsx
2025-05-12 15:28:39,461 - INFO - Archivo 'C:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\data\2_processed\ULTIMO_S&P500_final.xlsx' cargado con forma (2932, 132).
2025-05-12 15:28:39,462 - INFO - Columnas del DataFrame original: ['MoM_PRICE_Australia_10Y_Bond_bond', 'log_diff_PRICE_Australia_10Y_Bond_bond', 'rolling_var_PRICE_Australia_10Y_Bond_bond', '3M_change_PRICE_Australia_10Y_Bond_bond', 'zscore_PRICE_Australia_10Y_Bond_bond', 'MoM_PRICE_Japan_10Y_Bond_bond', 'YoY_PRICE_Japan_10Y_Bond_bond', 'log_diff_PRICE_Japan_10Y_Bond_bond', 'rolling_

In [13]:
!pip install lightgbm
!pip install xgboost


In [1]:
# Paso 7: Entrenamiento de modelos
print("Ejecutando paso 7: Entrenamiento de modelos")
%run ../pipelines/ml/step_7_0_train_models.py

Ejecutando paso 7: Entrenamiento de modelos


c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\venv_MValores\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-12 17:36:43,695 - INFO - Usando el archivo más reciente: C:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\data\3_trainingdata\ULTIMO_S&P500_final_FPI.xlsx
2025-05-12 17:36:43,698 - INFO - Usando horizonte de 20 días para forecast_period=1MONTH
2025-05-12 17:36:46,184 - INFO - Datos leídos y ordenados por fecha.
2025-05-12 17:36:46,193 - INFO - Se han imputado los valores NaN e inf (ffill y relleno con 0).
2025-05-12 17:36:46,226 - INFO - Tiempo de carga y preprocesamiento: 2.53s
2025-05-12 17:36:46,226 - INFO - Realizando división temporal por años (Year<=2022 para train+val)
2025-05-12 17:36:46,232 - INFO - 

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002116 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10385
[LightGBM] [Info] Number of data points in the train set: 356, number of used features: 106
[LightGBM] [Info] Start training from score 1974.842415
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

2025-05-12 17:46:26,607 - INFO - [LightGBM] Trial 0: RMSE=671.8982, Time=2.05s
[I 2025-05-12 17:46:26,609] Trial 0 finished with value: 671.8982284880136 and parameters: {'learning_rate': 0.004151639030613823, 'max_depth': 6, 'n_estimators': 597}. Best is trial 0 with value: 671.8982284880136.


Did not meet early stopping. Best iteration is:
[597]	valid_0's rmse: 837.482	valid_0's l2: 701376
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000718 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10385
[LightGBM] [Info] Number of data points in the train set: 356, number of used features: 106
[LightGBM] [Info] Start training from score 1974.842415
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

2025-05-12 17:47:24,190 - INFO - [LightGBM] Modelo refinado localmente en 0.22s


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

2025-05-12 17:47:24,214 - INFO - [LightGBM] Training predictions generadas en 0.02s (RMSE=1686.1045)
2025-05-12 17:47:24,223 - INFO - [LightGBM] Predicciones en Evaluacion generadas en 0.01s (RMSE=346.6524)
2025-05-12 17:47:24,240 - INFO - [LightGBM] Predicciones en Test generadas en 0.01s (RMSE=920.5975)
2025-05-12 17:47:24,263 - INFO - [LightGBM] Modelo refinado guardado en 0.02s: C:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\models\lightgbmtrial12.pkl
c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\venv_MValores\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\venv_MValores\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBM

✅ Entrenamiento completado para 5 algoritmos
✅ Visualizaciones generadas en: C:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\data\4_results\charts
✅ Modelos guardados en: C:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\models
✅ Predicciones consolidadas en: C:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\data\4_results\all_models_predictions.csv
✅ Tiempo total de ejecución: 2177.43s


In [2]:
# Paso 8: Preparación de salida
print("Ejecutando paso 8: Preparación de salida")
%run ../pipelines/ml/step_7_5_ensemble.py


2025-05-13 08:45:16,191 - INFO - Iniciando creación de ensamble greedy...
2025-05-13 08:45:16,209 - INFO - Modelo cargado: catboosttrial10
2025-05-13 08:45:16,228 - INFO - Modelo cargado: catboosttrial11
2025-05-13 08:45:16,241 - INFO - Modelo cargado: catboosttrial13
2025-05-13 08:45:16,253 - INFO - Modelo cargado: catboosttrial16
2025-05-13 08:45:16,268 - INFO - Modelo cargado: catboosttrial4
2025-05-13 08:45:16,283 - INFO - Modelo cargado: catboosttrial5
2025-05-13 08:45:16,305 - INFO - Modelo cargado: catboost_best
2025-05-13 08:45:16,330 - INFO - Modelo cargado: lightgbmtrial12


Ejecutando paso 8: Preparación de salida


2025-05-13 08:45:16,351 - INFO - Modelo cargado: lightgbmtrial15
2025-05-13 08:45:16,371 - INFO - Modelo cargado: lightgbmtrial18
2025-05-13 08:45:16,390 - INFO - Modelo cargado: lightgbmtrial19
2025-05-13 08:45:16,406 - INFO - Modelo cargado: lightgbmtrial7
2025-05-13 08:45:16,421 - INFO - Modelo cargado: lightgbm_best
2025-05-13 08:45:16,436 - INFO - Modelo cargado: mlptrial0
2025-05-13 08:45:16,454 - INFO - Modelo cargado: mlptrial1
2025-05-13 08:45:16,471 - INFO - Modelo cargado: mlptrial10
2025-05-13 08:45:16,487 - INFO - Modelo cargado: mlptrial11
2025-05-13 08:45:16,502 - INFO - Modelo cargado: mlptrial12
2025-05-13 08:45:16,520 - INFO - Modelo cargado: mlptrial14
2025-05-13 08:45:16,543 - INFO - Modelo cargado: mlp_best
2025-05-13 08:45:16,560 - INFO - Modelo cargado: svmtrial10
2025-05-13 08:45:16,579 - INFO - Modelo cargado: svmtrial11
2025-05-13 08:45:16,596 - INFO - Modelo cargado: svmtrial12
2025-05-13 08:45:16,614 - INFO - Modelo cargado: svmtrial16
2025-05-13 08:45:16,63

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002383 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25653
[LightGBM] [Info] Number of data points in the train set: 2261, number of used features: 109
[LightGBM] [Info] Start training from score 2862.772342
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

2025-05-13 08:45:40,736 - INFO - Modelo lightgbmtrial12 entrenado en 2.31s - RMSE: 1018.2621
2025-05-13 08:45:40,737 - INFO - Entrenando modelo base lightgbmtrial15 (9/32)


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

2025-05-13 08:45:52,613 - INFO - Modelo lightgbm_best entrenado en 1.28s - RMSE: 1014.8092
2025-05-13 08:45:52,614 - INFO - Entrenando modelo base mlptrial0 (14/32)


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

c:\Users\natus\Documents\Trabajo\PEDRO_PEREZ\Proyecto_Mercado_de_Valores\SP500_INDEX_Analisis\venv_MValores\Lib\site-packages\numpy\_core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


ValueError: Input X contains NaN.
MLPRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# Paso 9: Preparación de salida
print("Ejecutando paso 9: Preparación de salida")
%run ../pipelines/ml/step_8_prepare_output.py


In [ ]:
# Paso 10: Preparación de salida
print("Ejecutando paso 10: Preparación de salida")
%run ../pipelines/ml/step_9_backtest.py

print("¡Pipeline ML completado!")

In [ ]:
# Paso 11: Preparación de salida
print("Ejecutando paso 11: Preparación de salida")
%run ../pipelines/ml/step_10_inference.py
